In [28]:
import pandas as pd

df = pd.read_csv("Telco-Customer-Churn.csv")
df.head()

df.columns

Index(['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')

In [29]:
# Checking for mising vals

df.isnull().sum()

,0
customerID,0
gender,0
SeniorCitizen,0
Partner,0
Dependents,0
tenure,0
PhoneService,0
MultipleLines,0
InternetService,0
OnlineSecurity,0


In [30]:
# Convert TotalCharges to num

df["TotalCharges"] = pd.to_numeric(df["TotalCharges"], errors="coerce")

# drop rows w missing vals

df = df.dropna()

df.shape

df["Churn"] = df["Churn"].map({"Yes": 1, "No": 0})

df["Churn"].value_counts()

churn_rate = df["Churn"].mean()
print("Overall Churn Rate:", round(churn_rate * 100, 2), "%")

Overall Churn Rate: 26.58 %


Overall churn rate is 26.58%, highlighting that about 1/4 customers leave the service. Reducing the early churn is a signigicant opportunity to improve consumer lifetime.


In [31]:
# Find strongest churn driver

df.groupby("Contract")["Churn"].mean().sort_values(ascending=False)

,Churn
Contract,
Month-to-month,0.427097
One year,0.112772
Two year,0.028487


Here we see customers churn at around 43%, which is 1.6 times the platform average and 15 times higher than the 2 year contracts.

In [32]:
# Create Tenure buckets to check

df["TenureBucket"] = pd.cut(
  df["tenure"],
  bins= [0, 12, 24, 48, 72],
  labels=["0-12", "12-24", "24-48", "48-72"]
)

df["TenureBucket"].value_counts()

df.groupby("TenureBucket")["Churn"].mean()

/tmp/ipython-input-1635456109.py:11: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby("TenureBucket")["Churn"].mean()


,Churn
TenureBucket,
0-12,0.476782
12-24,0.287109
24-48,0.203890
48-72,0.095132


Customers are churning nearly 2 times the platform average (26.6%) in their first 12 months at 47.7%.


# Key Insight: Early Lifecycle Risk
Customers in their first 12 months exhibit a churn rate of 47.7%, nearly double the platform average of 26.6%. Retention improves steadily with tenure, dropping to 9.5% among customers with 48+ months. This suggests that the primary retention challenge occurs during the onboarding and early engagement phase.

In [33]:
# check online security & tech support

df.groupby("OnlineSecurity")["Churn"].mean()

df.groupby("TechSupport")["Churn"].mean()

,Churn
TechSupport,
No,0.416475
No internet service,0.074342
Yes,0.151961


Customers without TechSupport exhibit a churn rate of 41.6%, compared to 15.2% among customers who use TechSupport. This suggests that adoption of high-value features significantly improves retention. Improving early feature activation during onboarding may reduce churn among first-year customers.

In [34]:
# Assume average monthly revenue per customer
avg_monthly_revenue = df["MonthlyCharges"].mean()

# Total customers
total_customers = len(df)

# Current churn customers
current_churn_customers = total_customers * df["Churn"].mean()

# If churn reduced by 5 percentage points
improved_churn_rate = df["Churn"].mean() - 0.05
improved_churn_customers = total_customers * improved_churn_rate

customers_saved = current_churn_customers - improved_churn_customers

# Revenue impact (1 month projection)
monthly_revenue_saved = customers_saved * avg_monthly_revenue

print("Customers saved (if churn reduced by 5%):", round(customers_saved))
print("Estimated monthly revenue preserved: $", round(monthly_revenue_saved, 2))

Customers saved (if churn reduced by 5%): 352
Estimated monthly revenue preserved: $ 22783.05


# Current-State (As-Is)



*   Overall churn: 26.6%
*   Early lifecycle churn (0–12 months): 47.7%
* Month-to-month churn: 42.7%
* Customers without TechSupport churn: 41.6%

Primary Risk Segment:
New customers on month-to-month contracts with low feature adoption.

Retention failure is concentrated in early lifecycle engagement and value realization.

# Future-State (To-Be)

Implement a structured early lifecycle engagement framework:


1.   Guided onboarding workflow
2.   Automated feature activation prompts (TechSupport, Security)
3. Engagement dashboard tracking:
* Activation rate
* 30-day retention
* Feature adoption rate
* Targeted contract upgrade incentives after activation milestone


# KPI Framework

Primary KPI:

* 30-day churn rate (target: reduce from 47.7% to 40%)

Secondary KPIs:

* Feature activation within 30 days

* Conversion to 1-year contracts

* Customer engagement score

Guardrails:

* Onboarding completion time

* Support ticket volume
